In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.6/339.6 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
!pip install langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 3.4 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
from langchain_groq import ChatGroq

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('/content/drive/MyDrive/travel_advisor_Paris_2024/.env'))
groq_api_key = os.environ['GROQ_API_KEY']
serper_api_key = os.environ["SERPER_API_KEY"]

In [ ]:
llm = ChatGroq(temperature=0.0, groq_api_key=os.environ["GROQ_API_KEY"], model_name="llama3-70b-8192")

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
import requests
import json

from langchain.tools import tool
from langchain_community.document_loaders import WebBaseLoader


class SearchTools:

  @tool('search hotels')
  def search_hotels(query: str) -> str:
    """
    Use this tool to search the internet for the schedule of 2024 Paris Olympics competitions.
    """
    return SearchTools.search(f"site:https://www.hotels.com/ {query}", limit=5)

  @tool('open page')
  def open_page(url: str) -> str:
    """
    Use this tool to open a webpage and get the content.
    """
    loader = WebBaseLoader(url)
    return loader.load()


  def search(query, limit=5):

    url = "https://google.serper.dev/search"
    payload = json.dumps({
      "q": query,
      "num": limit,
    })
    headers = {
      'X-API-KEY': os.getenv("SERPER_API_KEY"),
      'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    results = response.json()['organic']

    string = []
    for result in results:
      string.append(f"{result['title']}\n{result['snippet']}\n{result['link']}\n\n")

    return f"Search results for '{query}':\n\n" + "\n".join(string)

In [ ]:
local_guide = Agent(
    role="Paris tour guide",
    goal="Recommend local attractions including 'hidden gems' and local experiences to a visitor according to"
          "their {interests}, avoid overcrowded tourist areas",
    backstory = "You have an extensive knowledge of Paris attractions and can give visitors"
                "detailed recommendations of places to visit taking their {interests} into consideration. Your recommendations always"
                "include lesser known attractions ('hidden gems') that allow the visitors"
                "to avoid crowded toursit areas and experience the authentic side of Paris.",
    allow_delegation=False,
    max_iter=5,
    llm=llm,
	  verbose=True
)

In [ ]:
hotel_guide = Agent(
    role="Expert on finding best hotels in central Paris, France",
    goal="Suggest best hotels for a visitor in Paris, France",
    backstory="You have an extensive knowledge of hotels in central Paris and can give best hotel recommendations"
              "to a visitor according to their budget.",
    allow_delegation=False,
    max_iter=5,
    llm=llm,
    verbose=True
)

In [ ]:
travel_planner = Agent(
        role="Amazing travel planner",
        goal="Create travel itineraries for travellers to Paris for {days} days"
             "including the best 'hidden gems' attractions and public transportation near each of them.",
        backstory="You are a specialist in travel planning with decades of experience."
                   "You provide a detailed travel itinerary for number of {days} days,"
                   "that includes the recommendations of the Paris tour guide",
        max_iter=5,
        llm=llm,
        verbose=True)

In [ ]:
recommend = Task(
    description=(
        "1. Recommend local attractions for a visitor according to their"
            "{interests}. Prioritize lesser known attractions that are away from tourist crowds."
        "2. Recommend authentic Parisian experiences according to "
            "visitor's {interests}."
        "3. Recommend authentic local restaurants. "
        "4. Recommend enough attractions to fill {days} days with activities in the mornings and afternoons."
        "4. Recommend enough places to eat for {days} days for breakfast, lunch and dinner."
        "5. Recommend public transportation near every place on your list."
    ),
    expected_output="A list of items, where each item includes the name of a place in Paris, location of the place, public transportation nearby"
                    "and a short description of why this place is worth visiting",
    agent=local_guide,
)

In [ ]:
find_hotel = Task(
    description=(
        "1. Recommend hotels in Central Paris, France to a visitor."
        "2. Your recommendations must be divided into 3 groups according to price per night:"
        "less than $200/per night, $200 - $500/ per night and above $500 per night."
        "3. Each group must contain 5 recommended hotels."
        "4. Only recommend the hotels with rating 8.0 and above."
    ),
    expected_output="A list of 3 groups of hotels, grouped by the price per night. Each group must contain 5 recommendations. For each hotel"
                    "give the name and address.",
    tools=[SearchTools.search_hotels, SearchTools.open_page],
    agent=hotel_guide
)

In [ ]:
plan = Task(
    description = (
        "2. Present the recommendations of the Paris tour guide as a full {days}-day travel plan."
        "4. You MUST give public transportation nearb each attraction, restaurant or event in the suggested travel plan."
        "3. Append the hotel recommendations of the hotel guide at the bottom of the travel itinerary."

    ),
    expected_output = "Your final answer MUST be a complete day-by-day travel plan,"
                      "based on the output of the recommend task"
                      "formatted as markdown, displaying daily recommendations that include:"
                      """
                       1. At least two places to visit each day
                       2. Restaurants to eat breakfast, lunch and dinner
                       3. Public transportation near each place
                       4. The reason you picked each place, what makes them special
                       """
                      "At the bottom append the hotel recommendations given by the hotel guide.",
    agent=travel_planner,
    context=[recommend, find_hotel],

)

In [ ]:
crew = Crew(
    agents=[local_guide, hotel_guide, travel_planner],
    tasks=[recommend, find_hotel, plan],
    verbose=2,
    max_rpm=29
)

In [ ]:
result = crew.kickoff(inputs={"interests": "architecture, wine", "days": 5})

 [DEBUG]: == Working Agent: Paris tour guide
 [INFO]: == Starting Task: 1. Recommend local attractions for a visitor according to theirarchitecture, wine. Prioritize lesser known attractions that are away from tourist crowds.2. Recommend authentic Parisian experiences according to visitor's architecture, wine.3. Recommend authentic local restaurants. 4. Recommend enough attractions to fill 5 days with activities in the mornings and afternoons.4. Recommend enough places to eat for 5 days for breakfast, lunch and dinner.5. Recommend public transportation near every place on your list.


> Entering new CrewAgentExecutor chain...
I now can give a great answer!

Final Answer:

**Day 1**

* **Morning:** Visit the **Musée de la Vie Romantique** (16, rue Chaptal, 75009 Paris, Metro: Saint-Georges or Blanche) - a hidden gem showcasing art and artifacts from the Romantic era, with a beautiful courtyard and charming architecture.
* **Lunch:** Head to **Le Comptoir du Relais** (9, Carrefour de l'O